In [1]:
import matplotlib.pyplot as plt
from PIL import Image

import numpy as np
from numpy import asarray

import tensorflow as tf
import os
import pandas as pd
import random
import time

In [2]:
directory = 'UTKFace'
ds_len = 0 
for filename in os.listdir(directory):
    if filename.endswith(".jpg"):
        ds_len += 1
    else:
        print(filename)
        
        continue
ds_len

.ipynb_checkpoints


23708

In [4]:
from PIL import Image

def get_num_pixels(filepath):
    width, height = Image.open(filepath).size
    return width*height

print(get_num_pixels("crop_part1/100_1_0_20170110183726390.jpg.chip.jpg"))

FileNotFoundError: [Errno 2] No such file or directory: 'crop_part1/100_1_0_20170110183726390.jpg.chip.jpg'

In [3]:
random.seed(time.time())
allNums = random.sample(range(ds_len), ds_len)
sets = [0]*10
for i in range(10):
    sets[i] = sorted(allNums[((ds_len//10) * i) : ((ds_len//10) * (i+1))])
sets[0][:5]

[2, 13, 23, 24, 37]

In [4]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import RandomFlip, Input, RandomRotation, Conv2D, BatchNormalization, MaxPool2D, Dropout, Flatten, Dense

In [5]:
def linear_model():
    model = Sequential([
        Input((200,200,3)),

        RandomFlip("horizontal_and_vertical"),
        RandomRotation(0.2),
        
        Conv2D(64, (3,3), activation='relu'),
        BatchNormalization(),

        Conv2D(64, (3,3), activation='relu'),
        BatchNormalization(),

        MaxPool2D(pool_size=(2,2)),
        Dropout(0.25),

        Conv2D(64, (3,3), activation='relu'),
        BatchNormalization(),

        Conv2D(64, (3,3), activation='relu'),
        BatchNormalization(),

        Dropout(0.25),

        Flatten(),

        Dense(256, activation='relu'),
        BatchNormalization(),

        Dropout(0.25),

        Dense(1, activation='linear')                           
    ])
    model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mae'])
    return model

In [6]:
def get_subset(set_n):
    found_ind = 0
    df_subset = np.zeros((len(sets[set_n])* 200*200*3), dtype='float32').reshape(-1,200,200,3)
    df_output = np.zeros((len(sets[set_n])), dtype='int8').reshape(len(sets[set_n]), 1)
    for count, filename in enumerate(os.listdir(directory)):
        if count == sets[set_n][found_ind]:
            df_subset[found_ind] = asarray(Image.open(f'UTKFace/{filename}'))
            df_output[found_ind][0] = int(filename.split('_')[0])
            found_ind += 1
            if found_ind == len(sets[set_n]):
                break;
    return df_subset, df_output

In [115]:
len(sets[1])

2370

In [10]:
X, y = get_subset(1)
y

array([[100],
       [ 10],
       [ 10],
       ...,
       [  9],
       [  9],
       [  9]], dtype=int8)

In [74]:
for count, filename in enumerate(os.listdir('UTKFace')):
    if filename.endswith('.jpg'):
        if count < 5:
            print(count, filename)
        os.rename(f'UTKFace/{filename}', f'UTKFace/{filename[:-9]}')
    else:
        continue;

0 100_0_0_201701122135.jpg
1 100_0_0_201701122152.jpg


FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'UTKFace/100_0_0_201701122152.jpg' -> 'UTKFace/100_0_0_2017011'

In [15]:
for count, filename in enumerate(os.listdir('UTKFace')):
    if count > 5:
        break;
    print(count, filename)

0 100_0_0_20170112213500903.jpg
1 100_0_0_20170112215240346.jpg
2 100_1_0_20170110183726390.jpg
3 100_1_0_20170112213001988.jpg
4 100_1_0_20170112213303693.jpg
5 100_1_0_20170112215032192.jpg


In [7]:
ageModel = linear_model()
ageModel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_flip (RandomFlip)    (None, 200, 200, 3)       0         
                                                                 
 random_rotation (RandomRota  (None, 200, 200, 3)      0         
 tion)                                                           
                                                                 
 conv2d (Conv2D)             (None, 198, 198, 64)      1792      
                                                                 
 batch_normalization (BatchN  (None, 198, 198, 64)     256       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 196, 196, 64)      36928     
                                                                 
 batch_normalization_1 (Batc  (None, 196, 196, 64)     2

In [8]:
from sklearn.model_selection import train_test_split

In [10]:
def PreProcessData(df):
    retdf = np.zeros(len(df)*200*200*1, dtype='float32').reshape(len(df),200,200,1)
    for ind, img in enumerate(df):
        print(f'{ind}/{len(df)}')
        for i in range(200):
            for j in range(200):
                retdf[ind][i][j] = np.sum(img[i][j]) / 768.0
    

In [13]:
for i in range(10):
    X, y = get_subset(i)
    trainX, testX, trainY, testY = train_test_split(X, y, test_size=0.2)
    #trainX = PreProcessData(trainX); testX = PreProcessData(testX)
    ageModel.fit(trainX, trainY, epochs=5, batch_size=128, validation_data=(testX, testY))

KeyboardInterrupt: 

In [122]:
for count, filename in enumerate(os.listdir('UTKFace')):
    if filename.endswith('.jpg'):
        display(filename.split('_'))
    else:
        continue;


['100', '0', '0', '2017011.jpg']

['100', '0', '0', '201701122152.jpg']

['100', '1', '0', '201701101837.jpg']

['100', '1', '0', '201701122130.jpg']

['100', '1', '0', '201701122133.jpg']

...

['9', '1', '3', '20161220222856346.jpg']

['9', '1', '3', '20170104222949455.jpg']

['9', '1', '4', '20170103200637399.jpg']

['9', '1', '4', '20170103200814791.jpg']

['9', '1', '4', '20170103213057382.jpg']

In [141]:
# plt.imshow('UTKFace/100_0_0_20170112213500903.jpg')
# plt.imshow(asarray(Image.open('UTKFace/1_0_0_20161219161028662.jpg')))

In [25]:
a = asarray(Image.open('UTKFace/1_0_0_20161219161028662.jpg'))
b = np.zeros(200*200).reshape(200,200,1)
for i in range(len(a)):
    for j in range(200):
        b[i][j] = np.sum(a[i][j]) / 768.0
b

array([[[0.61067708],
        [0.62109375],
        [0.61328125],
        ...,
        [0.81380208],
        [0.81770833],
        [0.81770833]],

       [[0.61328125],
        [0.61588542],
        [0.59895833],
        ...,
        [0.80989583],
        [0.81380208],
        [0.81380208]],

       [[0.59765625],
        [0.59375   ],
        [0.56380208],
        ...,
        [0.80598958],
        [0.80598958],
        [0.80989583]],

       ...,

       [[0.578125  ],
        [0.58203125],
        [0.58203125],
        ...,
        [0.64322917],
        [0.64713542],
        [0.64713542]],

       [[0.5859375 ],
        [0.5859375 ],
        [0.5859375 ],
        ...,
        [0.64322917],
        [0.64322917],
        [0.64322917]],

       [[0.58984375],
        [0.58984375],
        [0.59114583],
        ...,
        [0.640625  ],
        [0.64453125],
        [0.64453125]]])

In [145]:
len(a)

200

In [ ]:
X, y = get_subset(i)
trainX, testX, trainY, testY = train_test_split(X, y, test_size=0.2)
ageModel.fit(trainX, trainY, epochs=5, batch_size=128, validation_data=(testX, testY))

Epoch 1/5


In [31]:
X, y = get_subset(i)
trainX, testX, trainY, testY = train_test_split(X, y, test_size=0.2)
trainX = PreProcessData(trainX); testX = PreProcessData(testX)
# ageModel.fit(trainX, trainY, validation_data=(testX, testY))

KeyboardInterrupt: 

In [13]:
age = linear_model()
age.load_weights('./checkpoints/age_check')
for count, filename in enumerate(os.listdir('UTKFace')):
    if count > 5: 
        break;
    print(count)
    img = asarray(Image.open(f'UTKFace/{filename}'))
    Image.show(img)
    print(age.predict(img))
    

ResourceExhaustedError: OOM when allocating tensor with shape[565504,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:RandomUniform]